In [3]:
import numpy as np
import decimal
import matplotlib.pyplot as plt
import sympy as sp

In [4]:
def intializeContext(significantFigs,rounding) :
    #Intialize the context with specified signFigs and rounding/chopping
    decimal.getcontext().prec = significantFigs 
    if( rounding ) :
        decimal.getcontext().rounding = decimal.ROUND_HALF_UP
    else :
        decimal.getcontext().rounding = decimal.ROUND_DOWN 

In [5]:
def string_to_function(expr_str, var_name='x', module='math'):
    """
    Convert a string expression into a callable function f(x).

    Parameters
    ----------
    expr_str : str
        The mathematical expression in terms of `var_name`, e.g. "cos(x)+x**2".
    var_name : str
        The variable name used in the expression (default 'x').
    module : str
        Which backend to use for numerical evaluation:
         - 'math'   -> Python math (fast, double precision floats)
         - 'mpmath' -> mpmath (arbitrary precision, use mp.mpf values)
         - 'numpy'  -> numpy (vectorized, arrays)
    Returns
    -------
    function
        A callable f(x) that evaluates the expression.
    """
    # 1) Create a symbolic variable
    x = sp.symbols(var_name)

    # 2) Parse the string into a SymPy expression (raises on invalid input)
    expr = sp.sympify(expr_str)

    # 3) Choose module for lambdify
    if module == 'mpmath':
        # use mpmath for higher precision arithmetic
        numeric_module = 'mpmath'
    elif module == 'numpy':
        numeric_module = 'numpy'
    else:
        numeric_module = 'math'   # default: Python math (float)

    # 4) Create a fast callable from the symbolic expression
    f = sp.lambdify(x, expr, modules=numeric_module)

    return f

In [6]:
def parse_exp(expr_str , var_name='x') :
    x = sp.symbols(var_name)
    expr = sp.sympify(expr_str)
    
    def decimal_func(x_decimal) :
        if not isinstance(x_decimal,decimal.Decimal) :
            x_decimal = decimal.Decimal(x_decimal)
            
        result = expr.evalf(decimal.getcontext().prec,subs={x:x_decimal})
        return decimal.Decimal(str(result))
    
    return decimal_func   

In [15]:
def fixed_point_iteration(x0,func_expr,tol=1e-7 ,max_iter=100 , significantFigs = 14 , rounding = True):
    intializeContext(significantFigs,rounding=rounding)
    x_root = decimal.Decimal(x0)
    relativeError = decimal.Decimal("Infinity")
    g = parse_exp(func_expr)
    iteration_details = []
    iter = 0 
    for i in range(max_iter) :
        x_root_old = x_root
        x_root = g(x_root_old)
        if x_root != 0 and i != 0 :
            relativeError = abs(x_root-x_root_old)
            
        if func_expr != "" :
            expr_eval_str = f"{func_expr.replace('x', str(x_root_old))} = {x_root}"
        else:
            expr_eval_str = str(x_root)    
            
        detials = {"iteration" : i+1,"x_root" : x_root , "Relative Error" : relativeError , "Evaluation" : expr_eval_str}
        
        iteration_details.append(detials)
        if relativeError <= tol :
            break 
    return x_root, iteration_details
        

In [14]:
expr = "cos(x)"               # The user inputs this
g = parse_exp(expr)      # Convert to decimal-based function

# Run fixed point
root , details = fixed_point_iteration(
    g,
    x0="0.5",
    tol=decimal.Decimal("1e-10"),
    significantFigs=4,
    rounding=True
)

print("Function:", expr)
print("Approx root =", root)
print("Details of iterations:")
for detail in details:
    print(detail)

Function: cos(x)
Approx root = 0.7391
Details of iterations:
{Decimal('0.8776'): Decimal('0.8776'), 'Relative Error': Decimal('Infinity')}
{Decimal('0.6390'): Decimal('0.6390'), 'Relative Error': Decimal('0.2386')}
{Decimal('0.8027'): Decimal('0.8027'), 'Relative Error': Decimal('0.1637')}
{Decimal('0.6948'): Decimal('0.6948'), 'Relative Error': Decimal('0.1079')}
{Decimal('0.7682'): Decimal('0.7682'), 'Relative Error': Decimal('0.0734')}
{Decimal('0.7192'): Decimal('0.7192'), 'Relative Error': Decimal('0.0490')}
{Decimal('0.7523'): Decimal('0.7523'), 'Relative Error': Decimal('0.0331')}
{Decimal('0.7301'): Decimal('0.7301'), 'Relative Error': Decimal('0.0222')}
{Decimal('0.7451'): Decimal('0.7451'), 'Relative Error': Decimal('0.0150')}
{Decimal('0.7350'): Decimal('0.7350'), 'Relative Error': Decimal('0.0101')}
{Decimal('0.7418'): Decimal('0.7418'), 'Relative Error': Decimal('0.0068')}
{Decimal('0.7373'): Decimal('0.7373'), 'Relative Error': Decimal('0.0045')}
{Decimal('0.7403'): Decim

In [16]:
root , details = fixed_point_iteration(
    "0.5",
    expr,
    tol=decimal.Decimal("1e-10"),
    significantFigs=4,
    rounding=True
)
print("Function:", expr)
print("Approx root =", root)
print("Details of iterations:")
for detail in details:
    print(detail)

Function: cos(x)
Approx root = 0.7391
Details of iterations:
{'iteration': 1, 'x_root': Decimal('0.8776'), 'Relative Error': Decimal('Infinity'), 'Evaluation': 'cos(0.5) = 0.8776'}
{'iteration': 2, 'x_root': Decimal('0.6390'), 'Relative Error': Decimal('0.2386'), 'Evaluation': 'cos(0.8776) = 0.6390'}
{'iteration': 3, 'x_root': Decimal('0.8027'), 'Relative Error': Decimal('0.1637'), 'Evaluation': 'cos(0.6390) = 0.8027'}
{'iteration': 4, 'x_root': Decimal('0.6948'), 'Relative Error': Decimal('0.1079'), 'Evaluation': 'cos(0.8027) = 0.6948'}
{'iteration': 5, 'x_root': Decimal('0.7682'), 'Relative Error': Decimal('0.0734'), 'Evaluation': 'cos(0.6948) = 0.7682'}
{'iteration': 6, 'x_root': Decimal('0.7192'), 'Relative Error': Decimal('0.0490'), 'Evaluation': 'cos(0.7682) = 0.7192'}
{'iteration': 7, 'x_root': Decimal('0.7523'), 'Relative Error': Decimal('0.0331'), 'Evaluation': 'cos(0.7192) = 0.7523'}
{'iteration': 8, 'x_root': Decimal('0.7301'), 'Relative Error': Decimal('0.0222'), 'Evaluatio

In [19]:
root, table = fixed_point_iteration(
    "1", "(x + 3/x)/2", tol=1e-10, significantFigs=25
)
print("Approx root =", root)
print("Details of iterations:")
for detail in table:
    print(detail)

Approx root = 1.732050807568877293527446
Details of iterations:
{'iteration': 1, 'x_root': Decimal('2.000000000000000000000000'), 'Relative Error': Decimal('Infinity'), 'Evaluation': '(1 + 3/1)/2 = 2.000000000000000000000000'}
{'iteration': 2, 'x_root': Decimal('1.750000000000000000000000'), 'Relative Error': Decimal('0.250000000000000000000000'), 'Evaluation': '(2.000000000000000000000000 + 3/2.000000000000000000000000)/2 = 1.750000000000000000000000'}
{'iteration': 3, 'x_root': Decimal('1.732142857142857142857143'), 'Relative Error': Decimal('0.017857142857142857142857'), 'Evaluation': '(1.750000000000000000000000 + 3/1.750000000000000000000000)/2 = 1.732142857142857142857143'}
{'iteration': 4, 'x_root': Decimal('1.732050810014727540504108'), 'Relative Error': Decimal('0.000092047128129602353035'), 'Evaluation': '(1.732142857142857142857143 + 3/1.732142857142857142857143)/2 = 1.732050810014727540504108'}
{'iteration': 5, 'x_root': Decimal('1.732050807568877295254354'), 'Relative Erro